In [4]:
import folium

m = folium.Map(location=[51.6, 5.0], zoom_start=9)
m.save("testkaart.html")
print("Kaart succesvol opgeslagen! Open 'testkaart.html' in je browser.")


ModuleNotFoundError: No module named 'folium'

In [1]:
import pandas as pd
import folium
from pyproj import Transformer

# Stap 1: Laad de dataset
df = pd.read_csv("WKP_Meetobjecten_Waterschap Brabantse Delta_2024_20250414172129.csv", sep=";", encoding="utf-8")

# Stap 2: Zet RD naar WGS84
transformer = Transformer.from_crs("EPSG:28992", "EPSG:4326", always_xy=True)
df[['lon', 'lat']] = df.apply(
    lambda row: pd.Series(transformer.transform(row['GeometriePuntX_RD'], row['GeometriePuntY_RD'])),
    axis=1
)

# Stap 3: Definieer voedselbossen (al in RD → omzetten naar lat/lon)
voedselbossen_rd = {
    "Vierhoeven Roosendaal": (94471, 408274),
    "Haverleij": (136282, 412233),
    "Kleindongen": (116648, 407281),
    "Wildernest": (131417, 383390)
}
voedselbossen_wgs = {
    naam: transformer.transform(x, y)
    for naam, (x, y) in voedselbossen_rd.items()
}

# Stap 4: Maak de kaart
m = folium.Map(location=[51.6, 5.0], zoom_start=9)

# Voeg meetpunten toe
for _, row in df.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=3,
        color='blue',
        fill=True,
        fill_opacity=0.7
    ).add_to(m)

# Voeg voedselbossen toe
for naam, (lon, lat) in voedselbossen_wgs.items():
    folium.Marker(
        location=[lat, lon],
        popup=naam,
        icon=folium.Icon(color='green', icon='leaf')
    ).add_to(m)

# Laat kaart zien (in Jupyter gebruik: m)
m.save("meetpunten_voedselbossen_kaart.html")
print("Kaart opgeslagen als 'meetpunten_voedselbossen_kaart.html'")


ModuleNotFoundError: No module named 'folium'